<a href="https://colab.research.google.com/github/Szibill6/Szibill6.github.io/blob/main/bmiproject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import sys
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F # Importálni a torch fv-t, mert az elu csaj ebben a modulban található
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from google.colab import files


# 1. lépés: CSV fájl betöltése
uploaded = files.upload()
file_name = list(uploaded.keys())[0]
data = pd.read_csv(file_name)

# 2. lépés: Adatok előfeldolgozása
# Üres cellák keresése és a sorok törlése
initial_row_count = data.shape[0]  # Eredeti sorok száma
data = data.dropna()  # Üres cellákat tartalmazó sorok törlése
deleted_rows = initial_row_count - data.shape[0]  # Törölt sorok száma
print(f"Törölt sorok száma az üres cellák miatt: {deleted_rows}.")

# Minden oszlop konvertálása float típusúra, kivéve a 'sex' és 'Category' oszlopokat
for col in data.columns:
    if col not in ['Sex', 'Category']:
        data[col] = pd.to_numeric(data[col], errors='coerce')

# Magasság oszlop skálázása
if 'Height' in data.columns:
    data['Height'] = data['Height'] * 100

# Kerekítés egy tizedesjegyre
data = data.round(1)

# Nullák keresése és a sorok törlése a tábla visszamutatása ELŐTT
initial_row_count_zeros = data.shape[0]  # Sorok száma a törlés előtt
data = data[(data != 0).all(axis=1)]
deleted_rows_zeros = initial_row_count_zeros - data.shape[0]  # Törölt sorok száma
print(f"A nullák miatt törölt sorok száma: {deleted_rows_zeros}.")

# Adatok megerősítéshez való megjelenítése
print("Feldolgozott adatok:")
print(data.head())
input("Nyomd meg az Entert a folytatáshoz...")

# 'sex' és 'category' kódolása numerikus értékekként
label_encoders = {}
for col in ['Sex', 'Category']:
    if col in data.columns:
        le = LabelEncoder()
        data[col] = le.fit_transform(data[col])
        data[col] = data[col] + 1  # Hozzáadunk 1-et az értékekhez, mert a 0-tól kiakadt a program
        label_encoders[col] = le

# Adatok felosztása jellemzőkre és címkékre
X = data.drop(columns=['Bodyfat (%)', 'Muscle mass (%)', 'Category'], errors='ignore')
y = data[['Bodyfat (%)', 'Muscle mass (%)', 'Category']]

# Felosztás tanító és teszthalmazra
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Jellemzők skálázása
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# 3. lépés: Neurális hálózat definiálása
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.fc1 = nn.Linear(X_train.shape[1], 128)
        self.fc2 = nn.Linear(128, 128)
        self.fc3 = nn.Linear(128, 64)  # Három kimenet: Testzsír, Izomtömeg, Kategória
        self.fc4 = nn.Linear(64, 3)  # Kimeneti réteg
        self.dropout = nn.Dropout(0.1)  # Dropout réteg hozzáadása 0.1 valószínűséggel, sokat próbáltam, ez a leghatékonyabb

    def forward(self, x):
        x = F.elu(self.fc1(x)) # torch.relu helyett F.elu
        x = self.dropout(x)
        x = F.elu(self.fc2(x)) # torch.relu helyett F.elu
        x = self.dropout(x)
        x = F.elu(self.fc3(x)) # torch.relu helyett F.elu
        x = self.dropout(x)
        x = self.fc4(x)
        return x

# Modell, veszteségfüggvény és optimalizáló inicializálása
model = NeuralNetwork()
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3) # Adam optimalizáló használata SGD helyett, gyorsabb lett a tanulás.

# Adatok konvertálása PyTorch tenzorokká
train_dataset = torch.utils.data.TensorDataset(torch.Tensor(X_train), torch.Tensor(y_train.values))
train_loader = torch.utils.data.DataLoader(train_dataset, shuffle=True, batch_size=32) # 1-ről 32-re emelem szintén a gyorsításért

# 4. lépés: Modell tanítása
for epoch in range(1000):
    for inputs, targets in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
    if epoch % 100 == 0:
        print(f"Epoch {epoch}, Veszteség: {loss.item():.4f}")

# 5. lépés: Modell tesztelése
model.eval()
with torch.no_grad():
    y_pred = model(torch.Tensor(X_test)).numpy()
    accuracy = 100 - np.mean(np.abs((y_pred - y_test.values) / y_test.values) * 100)
    print(f"Modell pontossága: {accuracy:.2f}%")



Saving teaching.csv to teaching (25).csv
Törölt sorok száma az üres cellák miatt: 0.
A nullák miatt törölt sorok száma: 1.
Feldolgozott adatok:
      Sex  Age  Weight  Height  Muscle mass (kg)  Muscle mass (%)  \
0  Female   19    43.5   158.8              19.4             44.6   
1  Female   22    49.4   156.2              21.7             43.8   
2  Female   23    52.2   165.1              22.5             43.2   
3  Female   22    58.1   162.6              23.2             40.0   
4  Female   20    53.5   160.0              23.7             44.2   

   Bodyfat (kg)  Bodyfat (%)         Category  
0           8.2         18.8           Sovány  
1          10.0         20.3  Normál testsúly  
2          11.2         21.4  Normál testsúly  
3          15.8         27.3  Normál testsúly  
4          10.5         19.6  Normál testsúly  
Nyomd meg az Entert a folytatáshoz...
Epoch 0, Veszteség: 754.8574
Epoch 100, Veszteség: 4.9332
Epoch 200, Veszteség: 5.8745
Epoch 300, Veszteség: 3.9621